In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge, Lasso, ElasticNet, BayesianRidge, HuberRegressor
from sklearn.ensemble import (
    RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, 
    GradientBoostingRegressor, BaggingRegressor, HistGradientBoostingRegressor
)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
import xgboost as xgb
import lightgbm as lgb
import optuna
import torch

# Try TabPFN + TabM
try:
    from tabpfn import TabPFNRegressor
    TABPFN_AVAILABLE = True
except ImportError:
    print("⚠️  TabPFN not available. Install with: pip install tabpfn")
    TABPFN_AVAILABLE = False

try:
    import tabm
    if hasattr(tabm, 'TabMRegressor'):
        TabMRegressor = tabm.TabMRegressor
        TABM_AVAILABLE = True
    elif hasattr(tabm, 'TabM'):
        TabMRegressor = tabm.TabM
        TABM_AVAILABLE = True
    else:
        print(f"⚠️  TabM available attributes: {[attr for attr in dir(tabm) if not attr.startswith('_')]}")
        TABM_AVAILABLE = False
except ImportError:
    print("⚠️  TabM not available. Install with: pip install git+https://github.com/yandex-research/tabm.git")
    TABM_AVAILABLE = False

print("📂 Loading data...")
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
X = train.drop([f'BlendProperty{i}' for i in range(1, 11)], axis=1)
y = train[[f'BlendProperty{i}' for i in range(1, 11)]]
X_test = test.drop(['ID'], axis=1)

kf = KFold(n_splits=5, shuffle=True)

print("🚀 Starting Level 1 training...")

# Level 1 models
level1_names = ['XGB']
if TABM_AVAILABLE:
    level1_names.append('TabM')
if TABPFN_AVAILABLE:
    level1_names.append('TabPFN')

level1_names.extend(['LGBM', 'RandomForest', 'ExtraTrees', 'HistGB', 'SVR'])

print(f"📊 Level 1 models: {', '.join(level1_names)}")

level1_oof = {name: np.zeros(y.shape) for name in level1_names}
level1_test = {name: np.zeros((X_test.shape[0], y.shape[1])) for name in level1_names}

for t in range(y.shape[1]):
    print(f"🎯 Training Level 1 for target BlendProperty{t+1}...")
    for fold, (tr_idx, val_idx) in enumerate(kf.split(X)):
        print(f"  Fold {fold+1}/5")
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx, t], y.iloc[val_idx, t]

        # XGBoost
        xgb_model = xgb.XGBRegressor(
            n_estimators=200, max_depth=6, learning_rate=0.1,
            subsample=0.8, colsample_bytree=0.8, verbosity=0, n_jobs=-1
        )
        xgb_model.fit(X_tr, y_tr)
        level1_oof['XGB'][val_idx, t] = xgb_model.predict(X_val)
        level1_test['XGB'][:, t] += xgb_model.predict(X_test) / kf.n_splits

        # LightGBM
        lgb_model = lgb.LGBMRegressor(
            n_estimators=200, max_depth=6, learning_rate=0.1,
            subsample=0.8, colsample_bytree=0.8, verbose=-1, n_jobs=-1
        )
        lgb_model.fit(X_tr, y_tr)
        level1_oof['LGBM'][val_idx, t] = lgb_model.predict(X_val)
        level1_test['LGBM'][:, t] += lgb_model.predict(X_test) / kf.n_splits

        # RandomForest
        rf = RandomForestRegressor(n_estimators=200, max_depth=10, n_jobs=-1)
        rf.fit(X_tr, y_tr)
        level1_oof['RandomForest'][val_idx, t] = rf.predict(X_val)
        level1_test['RandomForest'][:, t] += rf.predict(X_test) / kf.n_splits

        # ExtraTrees
        et = ExtraTreesRegressor(n_estimators=200, max_depth=10, n_jobs=-1)
        et.fit(X_tr, y_tr)
        level1_oof['ExtraTrees'][val_idx, t] = et.predict(X_val)
        level1_test['ExtraTrees'][:, t] += et.predict(X_test) / kf.n_splits

        # ✅ HistGradientBoosting
        histgb = HistGradientBoostingRegressor(max_iter=200, max_depth=6, learning_rate=0.1)
        histgb.fit(X_tr, y_tr)
        level1_oof['HistGB'][val_idx, t] = histgb.predict(X_val)
        level1_test['HistGB'][:, t] += histgb.predict(X_test) / kf.n_splits

        # ✅ SVR (can be slow)
        svr = SVR(kernel='rbf', C=1.0, epsilon=0.1)
        svr.fit(X_tr, y_tr)
        level1_oof['SVR'][val_idx, t] = svr.predict(X_val)
        level1_test['SVR'][:, t] += svr.predict(X_test) / kf.n_splits

        # TabM
        if 'TabM' in level1_names and TABM_AVAILABLE:
            try:
                tabm_model = TabMRegressor(device='cuda' if torch.cuda.is_available() else 'cpu')
                tabm_model.fit(X_tr.values, y_tr.values)
                level1_oof['TabM'][val_idx, t] = tabm_model.predict(X_val.values)
                level1_test['TabM'][:, t] += tabm_model.predict(X_test.values) / kf.n_splits
            except Exception as e:
                print(f"    Error with TabM: {e}")
                level1_names.remove('TabM')

        # TabPFN
        if 'TabPFN' in level1_names and TABPFN_AVAILABLE:
            try:
                tabpfn = TabPFNRegressor(device='cuda' if torch.cuda.is_available() else 'cpu')
                tabpfn.fit(X_tr, y_tr.values)
                level1_oof['TabPFN'][val_idx, t] = tabpfn.predict(X_val)
                level1_test['TabPFN'][:, t] += tabpfn.predict(X_test) / kf.n_splits
            except Exception as e:
                print(f"    Error with TabPFN: {e}")
                level1_names.remove('TabPFN')

level1_names = [name for name in level1_names if level1_oof[name].sum() != 0]

print("\n📊 Level 1 MAPE Scores:")
for name in level1_names:
    mape = mean_absolute_percentage_error(y, level1_oof[name])
    print(f"  {name}: {mape:.6f}")

# ---------------------------------------
# LEVEL 2 STACKING
# ---------------------------------------
print("\n🔄 Preparing Level 2 inputs...")
stack_X = np.concatenate([level1_oof[name] for name in level1_names], axis=1)
stack_X_test = np.concatenate([level1_test[name] for name in level1_names], axis=1)

level2_names = [
    'Ridge', 'Lasso', 'ElasticNet', 'BayesianRidge', 'Huber',
    'RandomForest_L2', 'ExtraTrees_L2', 'AdaBoost', 'GradientBoost',
    'Bagging', 'KNN', 'SVR', 'XGB_L2', 'LGBM_L2'
]

level2_oof = {name: np.zeros(y.shape) for name in level2_names}
level2_test = {name: np.zeros((X_test.shape[0], y.shape[1])) for name in level2_names}

print("\n🚀 Starting Level 2 training...")
for t in range(y.shape[1]):
    print(f"🎯 Level 2 for BlendProperty{t+1}...")
    for fold, (tr_idx, val_idx) in enumerate(kf.split(stack_X)):
        print(f"  Fold {fold+1}/5")
        X_tr, X_val = stack_X[tr_idx], stack_X[val_idx]
        y_tr, y_val = y.iloc[tr_idx, t], y.iloc[val_idx, t]

        Ridge(alpha=1.0).fit(X_tr, y_tr)
        level2_oof['Ridge'][val_idx, t] = Ridge(alpha=1.0).fit(X_tr, y_tr).predict(X_val)
        level2_test['Ridge'][:, t] += Ridge(alpha=1.0).fit(X_tr, y_tr).predict(stack_X_test) / kf.n_splits

        lasso = Lasso(alpha=0.1, max_iter=1000)
        lasso.fit(X_tr, y_tr)
        level2_oof['Lasso'][val_idx, t] = lasso.predict(X_val)
        level2_test['Lasso'][:, t] += lasso.predict(stack_X_test) / kf.n_splits

        elastic = ElasticNet(alpha=0.1, max_iter=1000)
        elastic.fit(X_tr, y_tr)
        level2_oof['ElasticNet'][val_idx, t] = elastic.predict(X_val)
        level2_test['ElasticNet'][:, t] += elastic.predict(stack_X_test) / kf.n_splits

        bayesian = BayesianRidge()
        bayesian.fit(X_tr, y_tr)
        level2_oof['BayesianRidge'][val_idx, t] = bayesian.predict(X_val)
        level2_test['BayesianRidge'][:, t] += bayesian.predict(stack_X_test) / kf.n_splits

        huber = HuberRegressor()
        huber.fit(X_tr, y_tr)
        level2_oof['Huber'][val_idx, t] = huber.predict(X_val)
        level2_test['Huber'][:, t] += huber.predict(stack_X_test) / kf.n_splits

        rf2 = RandomForestRegressor(n_estimators=50)
        rf2.fit(X_tr, y_tr)
        level2_oof['RandomForest_L2'][val_idx, t] = rf2.predict(X_val)
        level2_test['RandomForest_L2'][:, t] += rf2.predict(stack_X_test) / kf.n_splits

        et2 = ExtraTreesRegressor(n_estimators=50)
        et2.fit(X_tr, y_tr)
        level2_oof['ExtraTrees_L2'][val_idx, t] = et2.predict(X_val)
        level2_test['ExtraTrees_L2'][:, t] += et2.predict(stack_X_test) / kf.n_splits

        ada = AdaBoostRegressor(n_estimators=50)
        ada.fit(X_tr, y_tr)
        level2_oof['AdaBoost'][val_idx, t] = ada.predict(X_val)
        level2_test['AdaBoost'][:, t] += ada.predict(stack_X_test) / kf.n_splits

        gb = GradientBoostingRegressor(n_estimators=100)
        gb.fit(X_tr, y_tr)
        level2_oof['GradientBoost'][val_idx, t] = gb.predict(X_val)
        level2_test['GradientBoost'][:, t] += gb.predict(stack_X_test) / kf.n_splits

        bag = BaggingRegressor(n_estimators=50)
        bag.fit(X_tr, y_tr)
        level2_oof['Bagging'][val_idx, t] = bag.predict(X_val)
        level2_test['Bagging'][:, t] += bag.predict(stack_X_test) / kf.n_splits

        knn = KNeighborsRegressor(n_neighbors=5)
        knn.fit(X_tr, y_tr)
        level2_oof['KNN'][val_idx, t] = knn.predict(X_val)
        level2_test['KNN'][:, t] += knn.predict(stack_X_test) / kf.n_splits

        svr2 = SVR(kernel='rbf')
        svr2.fit(X_tr, y_tr)
        level2_oof['SVR'][val_idx, t] = svr2.predict(X_val)
        level2_test['SVR'][:, t] += svr2.predict(stack_X_test) / kf.n_splits

        xgb2 = xgb.XGBRegressor()
        xgb2.fit(X_tr, y_tr)
        level2_oof['XGB_L2'][val_idx, t] = xgb2.predict(X_val)
        level2_test['XGB_L2'][:, t] += xgb2.predict(stack_X_test) / kf.n_splits

        lgb2 = lgb.LGBMRegressor()
        lgb2.fit(X_tr, y_tr)
        level2_oof['LGBM_L2'][val_idx, t] = lgb2.predict(X_val)
        level2_test['LGBM_L2'][:, t] += lgb2.predict(stack_X_test) / kf.n_splits

print("\n📊 Level 2 MAPE Scores:")
level2_scores = {name: mean_absolute_percentage_error(y, level2_oof[name]) for name in level2_names}
for name, mape in level2_scores.items():
    print(f"  {name}: {mape:.6f}")

print("\n🔍 Starting Level 3 dynamic ensemble...")

TOP_K = min(6, len(level2_names))
top_models = sorted(level2_scores, key=level2_scores.get)[:TOP_K]
print(f"📊 Top Level 2 models: {top_models}")

def objective(trial):
    w = np.array([trial.suggest_float(f'w_{m}', 0.0, 1.0) for m in top_models])
    w /= np.sum(w)
    final_pred = sum(w[i] * level2_oof[top_models[i]] for i in range(len(top_models)))
    return mean_absolute_percentage_error(y, final_pred)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200, show_progress_bar=True)

w = np.array([study.best_params[f'w_{m}'] for m in top_models])
w /= np.sum(w)

final_pred = sum(w[i] * level2_oof[top_models[i]] for i in range(len(top_models)))
final_test = sum(w[i] * level2_test[top_models[i]] for i in range(len(top_models)))
final_mape = mean_absolute_percentage_error(y, final_pred)

print(f"\n✅ Final Ensemble MAPE: {final_mape:.6f}")

submission = pd.DataFrame(final_test, columns=[f'BlendProperty{i}' for i in range(1, 11)])
submission.insert(0, 'ID', test['ID'])
submission.to_csv(f'submission_final_ensemble_last_stand.csv', index=False)
print("💾 Saved: submission_final_ensemble.csv")


📂 Loading data...
🚀 Starting Level 1 training...
📊 Level 1 models: XGB, TabM, TabPFN, LGBM, RandomForest, ExtraTrees, HistGB, SVR
🎯 Training Level 1 for target BlendProperty1...
  Fold 1/5
    Error with TabM: TabM.__init__() missing 1 required keyword-only argument: 'd_out'
    Error with TabPFN: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty2...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty3...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty4...
  Fold 1/5
  Fold 2/5
  Fold 3/5
  Fold 4/5
  Fold 5/5
🎯 Training Level 1 for target BlendProperty5...
  Fold 1/5
  Fold 

[I 2025-07-23 09:46:55,394] A new study created in memory with name: no-name-41af92d3-ff43-4d48-bf27-685417dab2ee



📊 Level 2 MAPE Scores:
  Ridge: 0.932206
  Lasso: 1.023738
  ElasticNet: 1.164840
  BayesianRidge: 0.929921
  Huber: 0.899805
  RandomForest_L2: 1.221228
  ExtraTrees_L2: 1.059653
  AdaBoost: 1.416212
  GradientBoost: 1.347286
  Bagging: 1.138906
  KNN: 2.313755
  SVR: 1.139877
  XGB_L2: 1.250419
  LGBM_L2: 1.133474

🔍 Starting Level 3 dynamic ensemble...
📊 Top Level 2 models: ['Huber', 'BayesianRidge', 'Ridge', 'Lasso', 'ExtraTrees_L2', 'LGBM_L2']


Best trial: 10. Best value: 0.912737:   5%|▌         | 10/200 [00:00<00:02, 92.35it/s]

[I 2025-07-23 09:46:55,407] Trial 0 finished with value: 0.9498447653913844 and parameters: {'w_Huber': 0.1179356672485874, 'w_BayesianRidge': 0.5519578041087901, 'w_Ridge': 0.5608202454156653, 'w_Lasso': 0.3370411432164836, 'w_ExtraTrees_L2': 0.6166483658728366, 'w_LGBM_L2': 0.4895858237597943}. Best is trial 0 with value: 0.9498447653913844.
[I 2025-07-23 09:46:55,417] Trial 1 finished with value: 0.9509436568434161 and parameters: {'w_Huber': 0.3364729110502177, 'w_BayesianRidge': 0.03736891871117598, 'w_Ridge': 0.7812774079828155, 'w_Lasso': 0.9493396278628392, 'w_ExtraTrees_L2': 0.8441331958025621, 'w_LGBM_L2': 0.3690392261418062}. Best is trial 0 with value: 0.9498447653913844.
[I 2025-07-23 09:46:55,428] Trial 2 finished with value: 0.968911497120429 and parameters: {'w_Huber': 0.16878261760345448, 'w_BayesianRidge': 0.23417365559993364, 'w_Ridge': 0.12801894580515905, 'w_Lasso': 0.4499840887303118, 'w_ExtraTrees_L2': 0.4961481987486348, 'w_LGBM_L2': 0.5045547375413285}. Best is

...

[I 2025-07-23 09:47:02,347] Trial 196 finished with value: 0.8998996355830563 and parameters: {'w_Huber': 0.6702545680368857, 'w_BayesianRidge': 0.07536253123720166, 'w_Ridge': 0.029964485245721314, 'w_Lasso': 0.18214263229853303, 'w_ExtraTrees_L2': 0.0009256108699747857, 'w_LGBM_L2': 0.0653957084455489}. Best is trial 193 with value: 0.8994442722677547.


Best trial: 193. Best value: 0.899444:  99%|█████████▉| 198/200 [00:07<00:00, 16.01it/s]

[I 2025-07-23 09:47:02,411] Trial 197 finished with value: 0.8997267206028636 and parameters: {'w_Huber': 0.6542622601899597, 'w_BayesianRidge': 0.08285043549902323, 'w_Ridge': 0.008341614515607599, 'w_Lasso': 0.17878572394884049, 'w_ExtraTrees_L2': 0.002438661841642098, 'w_LGBM_L2': 0.061446287882745365}. Best is trial 193 with value: 0.8994442722677547.
[I 2025-07-23 09:47:02,472] Trial 198 finished with value: 0.9007515366222363 and parameters: {'w_Huber': 0.6685664033292295, 'w_BayesianRidge': 0.09670582618853084, 'w_Ridge': 0.02777336192087624, 'w_Lasso': 0.2047911042257576, 'w_ExtraTrees_L2': 0.0005264957013044488, 'w_LGBM_L2': 0.0666474389215794}. Best is trial 193 with value: 0.8994442722677547.


Best trial: 193. Best value: 0.899444: 100%|██████████| 200/200 [00:07<00:00, 28.03it/s]

[I 2025-07-23 09:47:02,529] Trial 199 finished with value: 0.9005669887481538 and parameters: {'w_Huber': 0.655602229804877, 'w_BayesianRidge': 0.10127398766062101, 'w_Ridge': 0.02130775185318074, 'w_Lasso': 0.16819784055556242, 'w_ExtraTrees_L2': 0.0013730780579465536, 'w_LGBM_L2': 0.06861242449726004}. Best is trial 193 with value: 0.8994442722677547.

✅ Final Ensemble MAPE: 0.899444
💾 Saved: submission_final_ensemble.csv
